# Principal Component Analysis

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib widget
import common

In [2]:
data = common.loadFile("CleanedData")

Before we start we need to separate unused columns to the PCA. therefore, we are removing "RID" and "VISCODE" from the dataset. 
The column "DX" is our target and later will be represented on the Y axis of our graph

In [3]:
y =  data.loc[:,['DX']].values
patients = data.loc[:, ["RID"]].values

# We can now clean the dataset dropping what will not be used
x = data.drop(["RID", "VISCODE", "DX"], axis=1)

Once done time to scale the data bringing mean to 0 and the variance to 1

In [4]:
x = StandardScaler().fit_transform(x)
# data = data.sample(frac=1).reset_index(drop=True) # In case you need to randomize the lines

PCA Projection

In [5]:
pca = PCA(n_components=10)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data=principalComponents,  columns=[ 'principal component 1', 'principal component 2', 
                                                                'principal component 3', 'principal component 4',
                                                                'principal component 5', 'principal component 6',
                                                                'principal component 7', 'principal component 8',
                                                                'principal component 9', 'principal component 10' ])
finalDf = pd.concat([principalDf, data[['DX']]], axis = 1)
finalDf.head(5)
common.saveFile(finalDf, "PCAData")

In [6]:
plt.close()
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 Component PCA', fontsize = 20)


targets = [0, 1, 2]
colors = ['r', 'y', 'b']
for target, color in zip(targets, colors):
    indicesToKeep = finalDf['DX'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
    
ax.legend(['CN', 'MCI', 'Dementia'])
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
pca.explained_variance_ratio_


array([0.18543   , 0.05538921, 0.03611538, 0.02198693, 0.0152742 ,
       0.01361426, 0.01250852, 0.01247552, 0.01133669, 0.01055638])

In [8]:
plt.close()
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 4', fontsize = 15)
ax.set_ylabel('Principal Component 5', fontsize = 15)
ax.set_title('2 Component PCA', fontsize = 20)


targets = [0, 1, 2]
colors = ['r', 'y', 'b']
for target, color in zip(targets, colors):
    indicesToKeep = finalDf['DX'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 4']
               , finalDf.loc[indicesToKeep, 'principal component 5']
               , c = color
               , s = 50)
    
ax.legend(['CN', 'MCI', 'Dementia'])
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.close()
fig = plt.figure()
#ax = fig.add_subplot(1,1,1) 
ax = plt.axes(projection='3d')

ax.set_xlabel('Principal Component 1', fontsize=12)
ax.set_ylabel('Principal Component 2', fontsize=12)
ax.set_zlabel('Principal Component 3', fontsize=12)
ax.set_title('3 Component PCA', fontsize = 20)

targets = [0, 1, 2]
colors = ['r', 'y', 'b']
for target, color in zip(targets, colors):
    indicesToKeep = finalDf['DX'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , finalDf.loc[indicesToKeep, 'principal component 3']
               , c = color
               , s = 50)
    
ax.legend(['CN', 'MCI', 'Dementia'])
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
# Use only one feature
X = finalDf.drop("DX", axis=1).to_numpy().astype('int')
y = finalDf.loc[:,['DX']].to_numpy().astype('int').flatten()

# Split the data into training/testing sets
X_train = X[:-20]
X_test = X[-20:]

# Split the targets into training/testing sets
y_train = y[:-20]
y_test = y[-20:]

# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))



Coefficients: 
 [ 0.08701816  0.00169099 -0.02191746  0.03421871 -0.05062849 -0.01585486
 -0.07611896  0.01649684 -0.0352728  -0.01930593]
Mean squared error: 0.35
Coefficient of determination: -1.22


# Recursive Feature Elimination

In [12]:
# explore the number of selected features for RFE
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from matplotlib import pyplot

In [13]:
# get a list of models to evaluate
def get_models():
	models = dict()
	for i in range(2, len(X.T)):
		rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=i)
		model = DecisionTreeClassifier()
		models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
	return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores


# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))


>2 0.771 (0.026)
>3 0.742 (0.028)
>4 0.741 (0.028)
>5 0.739 (0.034)
>6 0.745 (0.028)
>7 0.746 (0.039)
>8 0.748 (0.030)
>9 0.741 (0.033)


In [24]:
# plot model performance for comparison
plt.close()
plt.boxplot(results, labels=names, showmeans=True)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Automatic Select the Number of Features

In [15]:
# create pipeline
rfe = RFECV(estimator=DecisionTreeClassifier(), min_features_to_select=3)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.740 (0.025)


In [16]:
estimator = DecisionTreeClassifier()
selector = RFECV(estimator, step=0.5, cv=cv)
selector = selector.fit(X_train, y_train)
selector.support_

array([ True, False, False, False, False, False, False, False, False,
       False])